[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alikn/coding_for_analytics/blob/main/llm/1_prompt_tuning.ipynb)

Some of the material in this notebook is inspired by Deeplearning.ai's [ChatGPT prompt engineering for developers](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/).

In [ ]:
%pip install -q openai

In [ ]:
api_key = '' # @param {type:"string"}

## Helfper functions for this notebook
The next cell includes helper functions which include functionality needed for this notebook. Just run the it and can skip to the next section.

In [ ]:
from openai import OpenAI


In [ ]:

MODEL = "gpt-4o-mini-2024-07-18"
client = OpenAI(api_key=api_key)

def get_completion(user_prompt, system_prompt=None, examples=None):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    if user_prompt:
        messages.append({"role": "user", "content": user_prompt})
    if examples:
        for example in examples:
            messages.append({"role": "user", "content": example[0]})
            messages.append({"role": "assistant", "content": example[1]})

    completion = client.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
get_completion("What is the meaning of life?")

## What is a large language model?

Large language models (LLMs) are powerful AI systems trained on vast amounts of text data to understand and generate human language. They operate by predicting the next word in a sequence, enabling them to generate coherent and contextually relevant text. LLMs like GPT-4 can perform a wide range of tasks, from answering questions and writing essays to translating languages and summarizing documents. Their capabilities stem from the extensive training on diverse datasets, which allows them to learn the nuances of language, including grammar, tone, and even some level of common sense reasoning.


## What is prompt tuning?
Prompt tuning is a technique used to optimize the performance of large language models on specific tasks by carefully crafting the input prompts. Instead of fine-tuning the entire model, which can be computationally expensive and time-consuming, prompt tuning focuses on modifying the prompts given to the model to elicit the desired responses. This approach leverages the pre-existing knowledge within the LLM, making it more efficient and adaptable to different applications. By adjusting the wording, structure, or context of the prompts, users can guide the model to generate more accurate or contextually appropriate outputs for a given task.

## Prompt tuning principles
### Give the LLM a persona
A quick way to set the stage for the LLM is to ask it to play a certain persona.

In [ ]:
prompt = """
You are a physicist. Describe a rainbow to a blind person in three sentences.
"""
get_completion(prompt)

In [ ]:
prompt = """
You are a poet. Describe a rainbow to a blind person in three sentences.
"""
get_completion(prompt)

### Write clear and specific instructions
Having detailed and specific instructions, reduces the chance of recieving irrelevant or incorrect responses. This does not mean that the prompt needs to be short.
#### Tactic 1: Use delimiters
Use delimeters when you want the model to perform a task on a certain piece of text.
- Triple quotes: """
- Triple backticks: ```
- Triple dashes: ---
- Angle brackets: <>

In [ ]:
prompt = """
Summarize the following text delimited by triple backticks in 3 sentences:
```
Python is a versatile and widely-used programming language known for its simplicity and readability, making it an ideal choice for both beginners and experienced developers. Its clean syntax emphasizes readability, which helps reduce the cost of program maintenance and allows programmers to express concepts in fewer lines of code than in languages like C++ or Java. Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming, providing flexibility in how developers approach problem-solving. It's also backed by a vast ecosystem of libraries and frameworks, such as NumPy for scientific computing, Django for web development, and TensorFlow for machine learning, which enable Python to be used in a wide range of applications, from data analysis and automation to web development and artificial intelligence.
```
"""
get_completion(prompt)

#### Tactic 2: Ask for structured output
If you want to use the output in a predefined way such as input to another function, it is helpful to ask the model to put the response in a certain structured format such as `json` or `HTML`.

In [ ]:
prompt = """
Generate a list of 5 courses thought in Menlo college this semester, their course numbers and their instructors.
Provide them in JSON format with the following keys: course, course_number, instructor.
The answer should not include any text outside of the JSON object.
"""
get_completion(prompt)

#### Tactic 3: Give instruction about edge cases
Sometimes, what you are asking of the model is not possible. If there is such a possibility, instruct the model to flag those.

In [ ]:
prompt = """
Solve the equation between the two curly braces and provide the solution in the form of x = value, y = value with no further text.
{{
    x + y = 10
    2x + y = 11
}}
"""
get_completion(prompt)

In [ ]:
prompt = """
Solve the equation between the two curly braces and provide the solution in the form of x = value, y = value with no further text.
{{
    Programming is fun!
}}
"""
get_completion(prompt)

In [ ]:
prompt = """
Solve the equation between the two curly braces and provide the solution in the form of x = value, y = value with no further text.
If there is no equation, simply write "No equation provided."
{{
    Programming is fun!
}}
"""
get_completion(prompt)

### In more complex tasks, ask for the chain of thought
Asking the model to just give the final response for complex tasks, can force the model to make incorrect guesses. On the other hand, if you ask for the details of reaching the conclusion, there is a higher chance that the response is sensible.

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not. 
The output should be either "Correct" or "Incorrect".

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not. 
Show the details of your work.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

### Give example responses to the model to follow
Writing clear and specific promopts, helps the model output to be more deterministic. We can take this one step further by giving the model examples to follow. There are two main ways for guiding the model using examples.

#### Give the example, in the body of the main message

In [ ]:
prompt = """
Your task is to pick up the colors in the given text.

<text>: A man with a blue jacket walking on a red carpet.

<colors>: blue, red 

<text>: A red car driving down a green road.
"""
response = get_completion(prompt)
print(response)

In [ ]:
system_prompt = """
Your task is to pick up the colors in the given text.
"""
examples = [["A man with a blue jacket walking on a red carpet.", "blue, red"]]
user_prompt = "A red car driving down a green road."
response = get_completion(system_prompt=system_prompt, user_prompt=user_prompt, examples=examples)
print(response)

## Model limitations: Hallucinations
LLMs are built to guess the next word which is sensible and can create content which is fictitious. LLMs are not aware of boundaries of their knowledge.

In [ ]:
prompt = f"""
Tell me about Menlo college's MSCW program.
"""
response = get_completion(prompt)
print(response)

### Exercise 1
We want to list a guitar on an online website. This is the description of the guitar we recieved from the manufacturer. For the website, we need to add a short one paragraph description and a list of specifications. The website is mostly visited by amateur musicians. Create the description and list of specifications.

In [ ]:
description = """
The EchoCaster Deluxe is a masterfully crafted electric guitar designed for musicians who demand both exceptional tone and striking aesthetics. Its solid body, made from carefully selected alder wood, offers a balanced tonal foundation with rich mids and resonant lows, ideal for a wide range of musical genres. The high-gloss cherry sunburst finish, accentuated by a flamed maple top, not only enhances the guitar’s visual appeal but also adds a layer of brightness to its sound, making each note stand out with clarity and precision. The lightweight design ensures comfort during long performances, while the double-cutaway body allows easy access to the higher frets, perfect for intricate solos.

At the heart of the EchoCaster Deluxe lies its versatile pickup configuration: a trio of custom-wound SonicFury pickups in an HSS (Humbucker-Single Coil-Single Coil) setup. The Alnico V humbucker in the bridge position delivers powerful, crunchy tones, perfect for rock and metal, while the single coils in the neck and middle positions offer warm, vintage tones suited for blues and jazz. With a 5-way selector switch and a push-pull tone knob for coil-splitting, this guitar provides a vast palette of sounds, allowing musicians to effortlessly switch between genres or mix tones to create their signature sound.

The EchoCaster Deluxe is not just about tone; it’s built for playability and reliability. The roasted maple neck, with its smooth satin finish and modern "C" profile, fits comfortably in the hand, making it ideal for fast runs and complex chord progressions. The ebony fingerboard, featuring 22 medium jumbo frets and pearloid dot inlays, ensures smooth playability and a luxurious feel. Grover locking tuners and a Wilkinson tremolo bridge provide rock-solid tuning stability, even during the most aggressive whammy bar dives. Whether you’re in the studio or on stage, the EchoCaster Deluxe is engineered to inspire, perform, and endure, making it the ultimate choice for serious guitarists.
"""
# Add your code here



### Exercise 2
After listing the guitar in the previous exercise, we have recieved 10 reviews and want to know what is the general sentiment of the customers. Create a function which recieves the review and returns the sentiment which can be `positive`, `negative` or `neutral`.

In [ ]:
reviews = [
    "The EchoCaster Deluxe is hands down the best guitar I’ve ever owned. The tone versatility is incredible, especially with the coil-splitting feature. I can easily switch from crunchy rock riffs to smooth bluesy leads without missing a beat. The neck feels great, and the cherry sunburst finish is absolutely stunning in person!",
    "I’ve been playing the EchoCaster Deluxe for a few weeks now, and it’s quickly become my go-to guitar. The HSS pickup configuration offers a wide range of tones, and the Wilkinson tremolo is super responsive. My only minor gripe is that it’s a bit heavier than I expected, but it’s a small price to pay for such amazing sound.",
    "This guitar is a dream for any player who loves both modern and vintage tones. The roasted maple neck is silky smooth, and the locking tuners keep it in perfect tune even after heavy tremolo use. The craftsmanship is top-notch, and the attention to detail is obvious in every aspect. I couldn’t be happier with my purchase!",
    "The EchoCaster Deluxe delivers on all fronts—looks, sound, and playability. The flamed maple top is gorgeous, and the custom-wound pickups offer a rich, full sound. It’s a bit pricier than other guitars I’ve considered, but the quality justifies the investment. I highly recommend it to anyone looking for a versatile and reliable instrument.",
    "I’ve been gigging with the EchoCaster Deluxe for the past month, and it’s performed flawlessly. The humbucker provides the punch I need for rock, while the single coils give me that sweet, clear tone for clean sections. The ebony fingerboard is a joy to play, and the satin-finished neck makes fast runs effortless. A fantastic guitar all around!",
    "As someone who plays a variety of genres, I appreciate how versatile the EchoCaster Deluxe is. The coil-splitting adds a lot of flexibility, and the pickups are very responsive. The only reason I’m not giving it 5 stars is that I had to adjust the action a bit out of the box, but now it plays like a dream.",
    "The EchoCaster Deluxe exceeded my expectations in every way. The sound quality is phenomenal, especially when using the bridge humbucker for heavier tones. The design is both beautiful and functional, and the guitar stays in tune no matter how much I use the tremolo. Definitely worth every penny!",
    "I was blown away by the build quality and tone variety of the EchoCaster Deluxe. The satin-finished neck is incredibly comfortable, and the pearloid inlays add a touch of class. The HSS configuration lets me cover a lot of musical ground, from jazz to hard rock. This guitar is a keeper!",
    "The EchoCaster Deluxe is a solid guitar with a lot to offer. The flamed maple top looks fantastic, and the pickups deliver a wide range of tones. It’s a bit on the heavier side, but the sound and playability make up for it. A great choice for anyone looking for a high-quality, versatile instrument.",
    "I’m in love with my EchoCaster Deluxe! The combination of the roasted maple neck and ebony fingerboard feels luxurious, and the tone possibilities are endless. Whether I’m playing clean jazz chords or ripping through a solo, this guitar delivers every time. Highly recommended for serious players!"
]

# Add your code here

